# ECSE 415: Final Project
Mathieu Geoffroy, 260986559
Ryan Reszetnik, 260948454


December 5th, 2023

In [38]:
import numpy as np
import cv2
import os
from ultralytics import YOLO
import torch
from collections import defaultdict

working_dir = os.path.curdir

In [39]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model = YOLO('yolov8n.pt').to(device)

# import YOLO labels from the dataset
yolo_labels = model.names

In [40]:
# Open the video file
def analyze_video(video_path, display=True, save=True):
    cap = cv2.VideoCapture(video_path)
    
    # get video name
    video_name = video_path.split('/')[-1].split('.')[0]
    
    out = cv2.VideoWriter(f'{video_name}_analyzed.mp4',cv2.VideoWriter_fourcc('m','p','4','v'), 30, (int(cap.get(3)),int(cap.get(4))))
    
    # Store the track history
    track_history = defaultdict(lambda: [])
    
    people_count = 0
    car_count = 0
    max_speed = 0
    
    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()
    
        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True)
    
            # Get the boxes, track IDs, class, for the frame
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            classes = results[0].boxes.cls.int().cpu().tolist()
    
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            
            # calculate the number of new people and cars
            for cls, track_id in zip(classes, track_ids):
                if yolo_labels[cls] == 'person' and track_id not in track_history:
                    people_count += 1
                if yolo_labels[cls] == 'car' and track_id not in track_history:
                    car_count += 1
                # calculate the speed by tracking stationary objects
                if yolo_labels[cls] == 'traffic light' or yolo_labels[cls] == 'stop sign' or yolo_labels[cls] == 'fire hydrant':
                    if len(track_history[track_id]) > 1:
                        last_point = track_history[track_id][-1]
                        second_last_point = track_history[track_id][-2]
                        distance = np.sqrt((last_point[0] - second_last_point[0])**2 + (last_point[1] - second_last_point[1])**2)
                        # convert pixels to meters
                        distance = distance * 0.0002645833
                        
                        # calculate speed in km/h
                        speed = distance * 3600
                        
                        max_speed = max(max_speed, speed)
                    
            
            # Display the number of people
            cv2.putText(annotated_frame, f"Number of people: {people_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # Display the number of cars
            cv2.putText(annotated_frame, f"Number of cars: {car_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # Display the speed
            cv2.putText(annotated_frame, f"Speed: {max_speed} km/h", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)     
    
            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)
    
                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
    
            # Display the annotated frame
            cv2.imshow(f'{video_name} Tracking', annotated_frame) if display else None
            
            # save the annotated frame to a new video
            out.write(annotated_frame) if save else None
    
            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
    
    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()

    return people_count, car_count, max_speed   

In [41]:
print(analyze_video(working_dir + '/mcgill_drive.mp4', display=False, save=True))
print(analyze_video(working_dir + '/st-catherines_drive.mp4', display=False, save=True))

0: 384x640 3 cars, 25.5ms
Speed: 2.2ms preprocess, 25.5ms inference, 15.3ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 3 cars, 12.7ms
Speed: 4.1ms preprocess, 12.7ms inference, 22.9ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 5 cars, 17.9ms
Speed: 2.3ms preprocess, 17.9ms inference, 13.6ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 5 cars, 14.3ms
Speed: 3.3ms preprocess, 14.3ms inference, 9.6ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 5 cars, 13.6ms
Speed: 3.4ms preprocess, 13.6ms inference, 19.5ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 5 cars, 15.9ms
Speed: 2.0ms preprocess, 15.9ms inference, 12.3ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 5 cars, 17.0ms
Speed: 2.3ms preprocess, 17.0ms inference, 12.6ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 5 cars, 14.7ms
Speed: 2.7ms preprocess, 14.7ms inference, 22.2ms postprocess per image at shape (1, 3, 384, 640)
0: 384x64

AttributeError: 'NoneType' object has no attribute 'int'